# Notebook to format gene and clinical data for the TCGA LUAD Cohort

Here we will choose the PanCancer version of the data.

The goal of this notebook is to format the data in a simple table whose rows are patient_id and column (gene mutation status + clinical)

Things to do:
 - [x] Download all the data from cBioPortal
 - [x] Download the gene data from cBioPortal by querying the list in 'gene_query.txt'
 - [x] Format the data
 - [ ] Apparently missing clinical values, but present in the GDC version, to check...
 - [ ] Check if everything is fine

In [1]:
import pandas as pd

In [2]:
# df_clinical = pd.read_csv("/home/val/workspaces/histotab/data/raw/TCGA_LUAD/luad_tcga_firehose_legacy/data_clinical_patient.txt", sep="\t", comment="#", skip_blank_lines=True)
# df_clinical = pd.read_csv("/home/val/workspaces/histotab/data/raw/TCGA_LUAD/luad_tcga_gdc/data_clinical_patient.txt", sep="\t", comment="#", skip_blank_lines=True)
df_clinical = pd.read_csv("/home/val/workspaces/histotab/data/raw/TCGA_LUAD/luad_tcga_pan_can_atlas_2018/data_clinical_patient.txt", sep="\t", comment="#", skip_blank_lines=True)

In [3]:
morhpology_col = "ICD_O_3_HISTOLOGY"
# morhpology_col = "MORPHOLOGY"

In [4]:
df_clinical.loc[:,morhpology_col].unique()

array(['8140/3', '8255/3', '8550/3', '8480/3', '8260/3', '8310/3',
       '8252/3', '8253/3', '8230/3', '8507/3', '8250/3', '8490/3', nan],
      dtype=object)

In [5]:
histology_map = {
    "8140/3": "Adenocarcinoma, NOS",
    "8255/3": "Adenocarcinoma with mixed subtypes",
    "8260/3": "Papillary adenocarcinoma, NOS",
    "8550/3": "Acinar cell carcinoma",
    "8480/3": "Mucinous adenocarcinoma",
    "8310/3": "Clear cell adenocarcinoma, NOS",
    "8252/3": "Bronchiolo-alveolar carcinoma, non-mucinous",
    "8253/3": "Invasive mucinous adenocarcinoma",
    "8230/3": "Solid carcinoma, NOS",
    "8507/3": "Invasive micropapillary carcinoma",
    "8250/3": "Lepidic adenocarcinoma",
    "8490/3": "Signet ring cell carcinoma",
    }

In [6]:
pattern_map = {
    "Lepidic adenocarcinoma": "Lepidic",
    "Bronchiolo-alveolar carcinoma, non-mucinous": "Lepidic",  # older term
    
    "Acinar cell carcinoma": "Acinar",

    "Papillary adenocarcinoma, NOS": "Papillary",

    "Solid carcinoma, NOS": "Solid",

    "Invasive micropapillary carcinoma": "Micropapillary",

    "Mucinous adenocarcinoma": "To drop",  # not part of 5 canonical patterns
    "Invasive mucinous adenocarcinoma": "To drop",  # not part of 5 canonical patterns

    "Clear cell adenocarcinoma, NOS": "To drop",  # not part of 5 canonical patterns
    "Signet ring cell carcinoma": "To drop",  # not part of 5 canonical patterns

    "Adenocarcinoma with mixed subtypes": "Mixed",  # optionally drop or keep as its own group
    "Adenocarcinoma, NOS": None  # too vague
}

In [7]:
df_clinical["HISTOLOGIC_SUBTYPE"] = df_clinical[morhpology_col].map(histology_map)
df_clinical["LUAD_PATTERN"] = df_clinical["HISTOLOGIC_SUBTYPE"].map(pattern_map)

In [8]:
df_clinical["LUAD_PATTERN"].unique()

array([None, 'Mixed', 'Acinar', 'To drop', 'Papillary', 'Lepidic',
       'Solid', 'Micropapillary', nan], dtype=object)

In [9]:
subtype_counts = df_clinical["LUAD_PATTERN"].value_counts().sort_values(ascending=False)

In [10]:
subtype_counts

LUAD_PATTERN
Mixed             107
Acinar             24
To drop            22
Lepidic            22
Papillary          21
Solid               5
Micropapillary      3
Name: count, dtype: int64

In [11]:
df_clinical = df_clinical[~df_clinical["LUAD_PATTERN"].isin(["To drop"])]

In [12]:
df_clinical.columns

Index(['PATIENT_ID', 'SUBTYPE', 'CANCER_TYPE_ACRONYM', 'OTHER_PATIENT_ID',
       'AGE', 'SEX', 'AJCC_PATHOLOGIC_TUMOR_STAGE', 'AJCC_STAGING_EDITION',
       'DAYS_LAST_FOLLOWUP', 'DAYS_TO_BIRTH',
       'DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS', 'ETHNICITY',
       'FORM_COMPLETION_DATE', 'HISTORY_NEOADJUVANT_TRTYN', 'ICD_10',
       'ICD_O_3_HISTOLOGY', 'ICD_O_3_SITE', 'INFORMED_CONSENT_VERIFIED',
       'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT', 'PATH_M_STAGE',
       'PATH_N_STAGE', 'PATH_T_STAGE', 'PERSON_NEOPLASM_CANCER_STATUS',
       'PRIMARY_LYMPH_NODE_PRESENTATION_ASSESSMENT', 'PRIOR_DX', 'RACE',
       'RADIATION_THERAPY', 'WEIGHT', 'IN_PANCANPATHWAYS_FREEZE', 'OS_STATUS',
       'OS_MONTHS', 'DSS_STATUS', 'DSS_MONTHS', 'DFS_STATUS', 'DFS_MONTHS',
       'PFS_STATUS', 'PFS_MONTHS', 'GENETIC_ANCESTRY_LABEL',
       'HISTOLOGIC_SUBTYPE', 'LUAD_PATTERN'],
      dtype='object')

In [13]:
path = "/home/val/workspaces/histotab/data/raw/TCGA_LUAD/TCGA_LUAD_PanCancer_gene_oncoprint.tsv"
df = pd.read_csv(path, sep="\t", index_col=0)

# Filter only mutation or CNA rows
df_filtered = df[df["track_type"].isin(["MUTATIONS", "CNA"])]

# Drop the 'track_type' column
df_filtered = df_filtered.drop(columns="track_type")

# Group by gene name and mark as 'MUT' if any value exists for a patient
df_binary = df_filtered.groupby(df_filtered.index).apply(lambda gene_df: gene_df.notnull().any()).T

# Convert boolean to 'MUT'/'WT'
df_binary = df_binary.replace({True: "MUT", False: "WT"})

# Rename index to patient_id
df_binary.index.name = "patient_id"

# Optional: view result
print(df_binary.shape)
print(df_binary.head())

(566, 21)
track_name   BRAF CDKN2A CTNNB1 EGFR KEAP1 KRAS MDM2 MET MGA MYC  ... NOTCH1  \
patient_id                                                        ...          
TCGA-55-6981  MUT     WT     WT  MUT    WT   WT   WT  WT  WT  WT  ...     WT   
TCGA-05-4402   WT    MUT     WT  MUT    WT   WT   WT  WT  WT  WT  ...     WT   
TCGA-50-6591   WT     WT     WT  MUT    WT   WT   WT  WT  WT  WT  ...     WT   
TCGA-38-6178   WT     WT     WT  MUT    WT   WT   WT  WT  WT  WT  ...     WT   
TCGA-86-8074   WT     WT     WT  MUT    WT   WT   WT  WT  WT  WT  ...     WT   

track_name   PIK3CA PTEN  RB1 RBM10 SETD2 SMAD4 SMARCA5 TERT TP53  
patient_id                                                         
TCGA-55-6981     WT   WT   WT    WT    WT    WT      WT   WT   WT  
TCGA-05-4402     WT   WT   WT    WT    WT    WT      WT  MUT  MUT  
TCGA-50-6591     WT   WT  MUT    WT    WT    WT      WT   WT  MUT  
TCGA-38-6178     WT   WT   WT   MUT    WT    WT      WT   WT  MUT  
TCGA-86-8074     WT  

In [14]:
df_binary.index.name = "patient_id"

In [15]:
mutations_counts = df_binary["EGFR"].value_counts().sort_values(ascending=False)

In [16]:
mutations_counts

EGFR
WT     482
MUT     84
Name: count, dtype: int64

In [17]:
# Set index to PATIENT_ID for the clinical dataframe
df_clinical = df_clinical.set_index("PATIENT_ID")

# Optionally rename index of the mutation DataFrame if not already done
df_binary.index.name = "patient_id"

# Merge on the index (patient ID)
df_merged = df_clinical.join(df_binary, how="inner")

# Keep only relevant clinical columns (you can modify this list)
relevant_columns = [
    "LUAD_PATTERN", "HISTOLOGIC_SUBTYPE", "AGE", "SEX",
    "OS_STATUS", "OS_MONTHS", "DSS_STATUS", "DSS_MONTHS",
    "DFS_STATUS", "DFS_MONTHS", "PFS_STATUS", "PFS_MONTHS"
]

# Combine with mutation data (gene columns come from df_binary)
final_columns = relevant_columns + list(df_binary.columns)
df_final = df_merged[final_columns]

# Optional: preview the result
print(df_final.shape)
print(df_final.head())


(544, 33)
             LUAD_PATTERN                  HISTOLOGIC_SUBTYPE   AGE     SEX  \
TCGA-05-4244         None                 Adenocarcinoma, NOS  70.0    Male   
TCGA-05-4249         None                 Adenocarcinoma, NOS  67.0    Male   
TCGA-05-4250         None                 Adenocarcinoma, NOS  79.0  Female   
TCGA-05-4382        Mixed  Adenocarcinoma with mixed subtypes  68.0    Male   
TCGA-05-4384        Mixed  Adenocarcinoma with mixed subtypes  66.0    Male   

               OS_STATUS  OS_MONTHS                  DSS_STATUS  DSS_MONTHS  \
TCGA-05-4244    0:LIVING   0.000000  0:ALIVE OR DEAD TUMOR FREE    0.000000   
TCGA-05-4249    0:LIVING  50.070684  0:ALIVE OR DEAD TUMOR FREE   50.070684   
TCGA-05-4250  1:DECEASED   3.978039                         NaN    3.978039   
TCGA-05-4382    0:LIVING  19.955946  0:ALIVE OR DEAD TUMOR FREE   19.955946   
TCGA-05-4384    0:LIVING  14.005326  0:ALIVE OR DEAD TUMOR FREE   14.005326   

                         DFS_STATUS  DFS

In [18]:
df_final

,LUAD_PATTERN,HISTOLOGIC_SUBTYPE,AGE,SEX,OS_STATUS,OS_MONTHS,DSS_STATUS,DSS_MONTHS,DFS_STATUS,DFS_MONTHS,...,NOTCH1,PIK3CA,PTEN,RB1,RBM10,SETD2,SMAD4,SMARCA5,TERT,TP53
TCGA-05-4244,None,"Adenocarcinoma, NOS",70.0,Male,0:LIVING,0.000000,0:ALIVE OR DEAD TUMOR FREE,0.000000,NaN,NaN,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
TCGA-05-4249,None,"Adenocarcinoma, NOS",67.0,Male,0:LIVING,50.070684,0:ALIVE OR DEAD TUMOR FREE,50.070684,NaN,NaN,...,WT,MUT,WT,WT,WT,WT,WT,WT,WT,WT
TCGA-05-4250,None,"Adenocarcinoma, NOS",79.0,Female,1:DECEASED,3.978039,NaN,3.978039,NaN,NaN,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
TCGA-05-4382,Mixed,Adenocarcinoma with mixed subtypes,68.0,Male,0:LIVING,19.955946,0:ALIVE OR DEAD TUMOR FREE,19.955946,1:Recurred/Progressed,10.980702,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,MUT
TCGA-05-4384,Mixed,Adenocarcinoma with mixed subtypes,66.0,Male,0:LIVING,14.005326,0:ALIVE OR DEAD TUMOR FREE,14.005326,NaN,NaN,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,MUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-17-Z045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,MUT
TCGA-17-Z031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WT,WT,MUT,MUT,WT,WT,WT,WT,WT,MUT
TCGA-17-Z049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WT,WT,WT,MUT,WT,MUT,WT,WT,WT,MUT
TCGA-17-Z060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WT,WT,WT,WT,MUT,WT,WT,WT,WT,WT


In [19]:
# Reset index and keep it as a column
df_final = df_final.reset_index()
df_final.index.name = "patient_id"

# Save to CSV with patient_id as the first column
df_final.to_csv("/home/val/workspaces/histotab/data/processed/tabular_data/tcga_luad_pancancer_combined_clinical_genes.csv", index=False)
